In [ ]:
# Import necessary libraries
%matplotlib inline
import nltk
from nltk.tokenize import word_tokenize
import re
from nltk.corpus import stopwords
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix
import seaborn as sns
# !pip install gensim
from sklearn.model_selection import cross_val_score, cross_val_predict, cross_validate
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
import pandas as pd
!pip install --only-binary=gensim gensim
!pip install --upgrade pip setuptools wheel
!pip install gensim
import gensim

In [ ]:
df = pd.read_csv('/content/SMSSpamCollection', delimiter='\t', names=['label', 'message'])
df.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
df.columns

Index(['label', 'message'], dtype='object')

In [ ]:
display(df.loc[15, 'label'])
display(df.loc[15, 'message'])

'spam'

'XXXMobileMovieClub: To use your credit, click the WAP link in the next txt message or click here>> http://wap. xxxmobilemovieclub.com?n=QJKGIGHJJGCBL'

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   label    5572 non-null   object
 1   message  5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [ ]:
df

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [ ]:
df = df.drop_duplicates()
df

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [ ]:
pd.options.mode.copy_on_write = True

In [ ]:
df['message'] = df['message'].str.replace(r'[.*,>]|http://wap\.', '', regex=True)
df['message']

,message
0,Go until jurong point crazy Available only in ...
1,Ok lar Joking wif u oni
2,Free entry in 2 a wkly comp to win FA Cup fina...
3,U dun say so early hor U c already then say
4,Nah I don't think he goes to usf he lives arou...
...,...
5567,This is the 2nd time we have tried 2 contact u...
5568,Will ü b going to esplanade fr home?
5569,Pity was in mood for that Soany other suggest...
5570,The guy did some bitching but I acted like i'd...


In [ ]:
df.isnull().sum()

,0
label,0
message,0


In [ ]:
df['label'].isin(['ham']).sum()

4516

In [ ]:
df['label'].isin(['spam']).sum()

653

In [ ]:
ham_df = df[df['label'] == 'ham']
spam_df = df[df['label'] == 'spam']

ham_sampled = ham_df.sample(n=653, random_state=42)

df_balanced = pd.concat([ham_sampled, spam_df]).reset_index(drop=True)

print(df_balanced['label'].value_counts())
df_balanced.head()

label
ham     653
spam    653
Name: count, dtype: int64


,label,message
0,ham,Sorry I can't help you on this
1,ham,How's my loverboy doing ? What does he do that...
2,ham,Sitting in mu waiting for everyone to get out ...
3,ham,Sure! I am driving but will reach my destinati...
4,ham,Sorry went to bed early nightnight


In [ ]:
df_balanced['label'].isin(['ham']).sum()

653

In [ ]:
df_balanced['label'].isin(['spam']).sum()

653

In [ ]:
print(df_balanced['label'].unique())

['ham' 'spam']


In [ ]:
df_balanced.loc[:,'label'] = df_balanced['label'].map({'spam': 1, 'ham': 0});

In [ ]:
df_balanced['label']

,label
0,0
1,0
2,0
3,0
4,0
...,...
1301,1
1302,1
1303,1
1304,1


In [ ]:
nltk.download('punkt_tab');
nltk.download('stopwords');
nltk.download('wordnet');
def preprocessing_text(token):

    tokens = word_tokenize(token.lower())
    stop_words = set(stopwords.words('english'));
    filtered_tokens = [token for token in tokens if token not in stopwords.words('english')]
    # lemmatizer = WordNetLemmatizer()
    stemmer = PorterStemmer()

    # lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
    stemmed_tokens = [stemmer.stem(word) for word in filtered_tokens]

    # Join the tokens back into a string for TFIVectore
    # processed_text = ' '.join(stemmed_tokens)

    return stemmed_tokens



[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
df_balanced.loc[12, 'message']

"When is school starting Where will you stay What's the weather like And the food Do you have a social support system like friends in the school All these things are important"

In [ ]:
df_balanced.loc[:,'message'] = df_balanced.loc[:,'message'].apply(preprocessing_text);

In [ ]:
df_balanced.loc[12, 'message']

['school',
 'start',
 'stay',
 "'s",
 'weather',
 'like',
 'food',
 'social',
 'support',
 'system',
 'like',
 'friend',
 'school',
 'thing',
 'import']

In [49]:
x = df_balanced['message'].values
y = df_balanced['label'].values

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
print(f"len x_train :  {len(x_train)}, len x_test : {len(x_test)}")
print(f"shape x_train :  {x_train.shape}, len y_train : {y_train.shape}")

len x_train :  1044, len x_test : 262
shape x_train :  (1044,), len y_train : (1044,)


In [51]:

model = gensim.models.Word2Vec(x_train, vector_size=100, window=5, min_count=1, workers=1, sg=0) #using CBOW Architecture for trainnig
modelt = gensim.models.Word2Vec(x_test, vector_size=100, window=5, min_count=1, workers=1, sg=0) #using skipGram Architecture for training if sg=1


In [53]:

texts = [' '.join(tokens) for tokens in x_train]

# Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

# Convert texts into sequence
x_train_seq = tokenizer.texts_to_sequences(texts)
x_train_pad = pad_sequences(x_train_seq, padding='post')

texts_test = [' '.join(tokens) for tokens in x_test]
x_test_seq = tokenizer.texts_to_sequences(texts_test)
x_test_pad = pad_sequences(x_test_seq, maxlen=x_train_pad.shape[1], padding='post')

# Create embedding matrix from Word2Vec
embedding_dim = 100
vocab_size = len(tokenizer.word_index) + 1
embedding_matrix = np.zeros((vocab_size, embedding_dim))

for word, i in tokenizer.word_index.items():
    if word in model.wv:
        embedding_matrix[i] = model.wv[word]

# Create LSTM model
model_lstm = Sequential()
model_lstm.add(Embedding(input_dim=vocab_size,
                         output_dim=embedding_dim,
                         weights=[embedding_matrix],
                         input_length=x_train_pad.shape[1],
                         trainable=False))
model_lstm.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model_lstm.add(Dense(1, activation='sigmoid'))  # for binary classification

model_lstm.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_lstm.summary()

# convert object into int
y_train = y_train.astype('int32')
y_test = y_test.astype('int32')

# آموزش مدل
model_lstm.fit(x_train_pad, y_train, epochs=5, batch_size=64, validation_data=(x_test_pad, y_test))


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)         │ ?                      │       361,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 361,800 (1.38 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 361,800 (1.38 MB)

Epoch 1/5
17/17 ━━━━━━━━━━━━━━━━━━━━ 13s 252ms/step - accuracy: 0.4679 - loss: 0.6933 - val_accuracy: 0.5115 - val_loss: 0.6931
Epoch 2/5
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 190ms/step - accuracy: 0.5042 - loss: 0.6931 - val_accuracy: 0.8588 - val_loss: 0.6805
Epoch 3/5
17/17 ━━━━━━━━━━━━━━━━━━━━ 6s 255ms/step - accuracy: 0.6839 - loss: 0.6128 - val_accuracy: 0.8282 - val_loss: 0.4164
Epoch 4/5
17/17 ━━━━━━━━━━━━━━━━━━━━ 4s 184ms/step - accuracy: 0.8097 - loss: 0.4798 - val_accuracy: 0.8626 - val_loss: 0.3592
Epoch 5/5
17/17 ━━━━━━━━━━━━━━━━━━━━ 5s 205ms/step - accuracy: 0.8497 - loss: 0.4156 - val_accuracy: 0.8626 - val_loss: 0.3450


In [59]:
loss_tr, accuracy_tr = model_lstm.evaluate(x_train_pad, y_train)
loss_te, accuracy_te = model_lstm.evaluate(x_test_pad, y_test)
print('training loss: {:.2f} , {:.2f} accuracy: \n test loss: {:.2f} , {:.2f} accuracy'.format(loss_tr, accuracy_tr, loss_te, accuracy_te))


33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.8494 - loss: 0.3952
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step - accuracy: 0.8554 - loss: 0.3603
training loss: 0.39 , 0.85 accuracy: 
 test loss: 0.34 , 0.86 accuracy


In [60]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred_prob = model_lstm.predict(x_test_pad)
y_pred = (y_pred_prob > 0.5).astype("int32")

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))


9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 164ms/step
              precision    recall  f1-score   support

           0       0.84      0.88      0.86       129
           1       0.88      0.84      0.86       133

    accuracy                           0.86       262
   macro avg       0.86      0.86      0.86       262
weighted avg       0.86      0.86      0.86       262

[[114  15]
 [ 21 112]]
